In [15]:
import time
import requests
from bs4 import BeautifulSoup
import pymongo

In [16]:
mc = pymongo.MongoClient()  # Connect to the MongoDB server using default settings
db = mc['strain_reviews']  # Use (or create) a database called 'election_predictions'
docs = db['review']  # Use (or create) a collection called 'docs'

In [57]:
docs.count_documents(filter={})

0

In [58]:
docs.drop()

## Here is how i want the scrape to work

    """LOS = [List of Strains]
        for s in LOS:
        url = "https://www.leafly.com/{}/reviews?page=".format(s)
        for i in range(1,100):
            d = get_reviews(url+str(i))
            st, rev = parse_docs(d)
            if len(st) > 0:
               append.(add to your dictionary)
            else:
                break"""

In [19]:
def table_check2(l):
    """This checks to see if a url is already in my database of reviews. 
    Only needs to be run in the get reviews function"""
    for url in l:
        if docs.find_one({'url':url}):
            None
        else:
            docs.insert_one({'url': url,
                 'html': requests.get(url).content
                 })

In [20]:
def get_reviews2(l):
    """Pass in a list of URL's and return them in a mongo db table as a dicitonary with 
    {'url', 'html'} and their corresponding values"""
    r = requests.get(l)
    html = (r.content)
    return html

In [21]:
def get_stars_list(d):
    stars = []
    for key, values in d.items():
        soup = BeautifulSoup(values, 'html.parser')
        tags = soup.select("div.div.stars")
        for t in tags:
            stars.append(t.attrs['style'])
    return stars

In [22]:
def star_int_conv(l):
    flat_stars= []
    star_num = []
    for s in l:
        star = int((s[6:].split(';')[0]).strip('px'))
        star_num.append((star/22))
    return star_num

In [23]:
def list_o_strains(i):
    LOS = []
    r = requests.get(i)
    soup2 = BeautifulSoup(r.content, 'html.parser')
    strains = soup2.find_all('a', class_="ga_Explore_Strain_Tile")
    for s in strains:
        LOS.append(s.attrs['href'])
    return LOS

In [24]:
def parse_docs2(d):
    """Parse the HTML docs that we have stored in a dictionary, return as a list.
    Also scrape and parse star rating for each review """
    strain_text= []
    star_rate = []
    user_name = []
    soup = BeautifulSoup(d, 'html.parser')
    revs = soup.find_all('p',class_='strain-review__text') 
    for r in revs:
        strain_text.append(r.text)
    tags = soup.select("div.div.stars")
    star = []
    for t in tags:
        star.append(t.attrs['style'])
    star_rate.append(star_int_conv(star))
    users = soup.find_all('div', class_='strain-review__title')
    for u in users:
        temp = u.find('h2')
        user_name.append(temp.text)
    return star_rate, strain_text, user_name

In [25]:
#Test works better, is updated!
def scraper_dummy1(los):
    los_urls = []
    rev_urls = []
    for s in los:
        url = "https://www.leafly.com{}/reviews?page=".format(s)
        print(url)
        for i in range(1,100):
            rev_urls=url+str(i)
            d = get_reviews2(rev_urls)
            star, reviews, users = parse_docs2(d)
            print(users)
            if len(star[0]) == 0:
                break

In [129]:
def scraper_dummy_test(los):
    los_urls = []
    rev_urls = []
    user_list_of_lists = []
    star_list_of_lists = []
    reviews_list_o_lists = []
    #grab URL landing pages for reviews on each strain.
    for s in los:
        url = "https://www.leafly.com{}/reviews?page=".format(s)
        strain_star_list = []
        strain_user_list = []
        strain_rev_list = []
        #get all reviews, stars, and users for each strain in list of strains
        for i in range(1,100):
            rev_urls=url+str(i)
            d = get_reviews2(rev_urls)
            star, reviews, users = parse_docs2(d)
            #return stars, strains, reviews, in one list, not list of lists for each page scraped. 
            #star_unlisted = 
            strain_star_list.extend([x for x in star[0]])
            strain_user_list.extend(users)
            strain_rev_list.extend(reviews)
            if len(star[0]) == 0:
                break
        #these are long single lists, not list of lists
        star_list_of_lists.append(strain_star_list)
        user_list_of_lists.append(strain_user_list)
        reviews_list_o_lists.append(strain_rev_list)
        #make into dictionaries
        for s in los:
            for i in range(len(star_list_of_lists)):
                user_revs =  dict(list(zip(user_list_of_lists[i], reviews_list_o_lists[i])))
                user_stars = dict(list(zip(user_list_of_lists[i], star_list_of_lists[i])))
        #create Pandas DF of each dictionary as well.
    return  user_revs, user_stars

In [126]:
test_strains = list_o_strains('https://www.leafly.com/explore')

In [127]:
test_strains[6]

'/hybrid/original-glue'

In [128]:
reviews_d, stars_d= scraper_dummy_test(test_strains[6:8])

IndexError: list index out of range

In [132]:
stars_d

{'hi2': 4.0,
 'MurderMyLove': 5.0,
 'JJMaxwell': 5.0,
 'greeneyeganja.girl': 4.0,
 'GreenPie': 5.0,
 'Ramoburo': 5.0,
 'kantheist': 5.0,
 'iamcherryface': 4.0,
 'GanjaGoddess410': 4.0,
 'ChemicalHazex': 5.0,
 'BobCantwell': 5.0,
 'BadBehavior89': 5.0,
 'KindGodess420': 5.0,
 'willir97': 5.0,
 'somacolorado': 5.0,
 'CanadaKush': 4.0,
 'John18': 5.0,
 'Pikarya': 5.0,
 'TSmoove': 4.0,
 'whitewidow81': 4.0,
 'allnaturalfitness': 5.0,
 'Flakzter': 5.0,
 'xSarahcate': 4.0,
 'StonerStig': 4.0,
 'bobross559': 5.0,
 'srubek': 4.0,
 'ImaFlyboy': 4.0,
 'ArthritisPain': 3.0,
 'Lumo': 4.0,
 'Reighly': 4.0,
 'BakinKush': 5.0,
 'laj821': 5.0,
 'hurdyhurdy': 4.0,
 'Dryland2012': 5.0,
 'highmonkey': 5.0,
 'kalamardesk': 5.0,
 'StonedHousewife': 5.0,
 'HudsonCole': 5.0,
 'RavenBanes': 5.0,
 'Kayley14': 4.0,
 'cmgunder': 5.0,
 'Spanish420': 5.0,
 'bdriskell99': 5.0,
 'DiscordedDiscord': 5.0,
 'inthewild': 5.0,
 'Deevaa': 5.0,
 'vito.genovese.7568': 4.0,
 'Aaro': 4.0,
 'artyfartyblah': 5.0,
 'googleymoo':

In [130]:
user_stars = dict(list(zip(users[0], stars[0])))

In [131]:
user_revs = dict(list(zip(users[0], reviews[0])))

In [73]:
user_revs

{'chronicallymedicated': '"Initial impression: My bud-tender said in the usual tone "what\'ll it be?" I chirped back, "surprise me, something new from the cannaseur shelf\' please". So I was handed a newer strain aptly named "Mother\'s Milk. The other one was Gorilla Glue #4 (GG)" (reviewed in this post). Needless to say, I was pretty excited about both of these. I sampled this one in a pre-rolled cone form, also rolled with "Raw Cones" thanks to..."',
 'MZ_PHD_in_420': '"SOON TO BE TOP !0 STRAIN OF ALL TIME!!!I have 40+ years experience growing and using cannabis, a masters degree in agri/horticulture from MSU and will break each review down as follows from my own personal experience with seeds or clones obtained directly from the original breeders. I received my clone directly from the person that entered it in the 2014 Michigan High Times Cannabis Cup, where it won for the second t..."',
 'gatorhunts420': '"Shhh don\'t tell anybody but I rolled my best shit from my last harvest of t